In [2]:
import cv2
import face_recognition
import pickle
import numpy as np

# Load known encodings
with open("../trained_model/encodings.pickle", "rb") as f:
    data = pickle.load(f)

known_face_encodings = data["encodings"]
known_face_names = data["names"]

# Start webcam
video_capture = cv2.VideoCapture(0)

if not video_capture.isOpened():
    print("❌ Error: Could not open webcam.")
    exit()

print("✅ Webcam started. Press 'q' to quit.")

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("❌ Failed to grab frame.")
        break

    # Resize frame for faster processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    # Convert from BGR to RGB
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Detect faces
    face_locations = face_recognition.face_locations(rgb_small_frame, model="cnn")

    # Encode faces
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        name = "Unknown"
        confidence_text = ""

        # Compare against known faces
        distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        if len(distances) > 0:
            best_match_index = np.argmin(distances)
            if distances[best_match_index] < 0.45:
                name = known_face_names[best_match_index]
                confidence = round((1 - distances[best_match_index]) * 100, 2)
                confidence_text = f"{confidence}%"

        # Scale back to original frame size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw face box
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        
        # Name above box
        cv2.putText(frame, name, (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # Accuracy below box
        if confidence_text:
            cv2.putText(frame, confidence_text, (left, bottom + 25),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # Show video
    cv2.imshow("Live Face Recognition", frame)

    # Quit on 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


✅ Webcam started. Press 'q' to quit.
